In [50]:
!pip install streamlit

  Using cached streamlit-1.48.1-py3-none-any.whl.metadata (9.5 kB)
  Using cached altair-5.5.0-py3-none-any.whl.metadata (11 kB)
  Using cached blinker-1.9.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached cachetools-6.1.0-py3-none-any.whl.metadata (5.4 kB)
  Using cached click-8.2.1-py3-none-any.whl.metadata (2.5 kB)
  Using cached pyarrow-21.0.0-cp312-cp312-win_amd64.whl.metadata (3.4 kB)
  Using cached tenacity-9.1.2-py3-none-any.whl.metadata (1.2 kB)
  Using cached toml-0.10.2-py2.py3-none-any.whl.metadata (7.1 kB)
  Using cached watchdog-6.0.0-py3-none-win_amd64.whl.metadata (44 kB)
  Using cached gitpython-3.1.45-py3-none-any.whl.metadata (13 kB)
  Using cached pydeck-0.9.1-py2.py3-none-any.whl.metadata (4.1 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached jsonschema-4.25.0-py3-none-any.whl.metadata (7.7 kB)
  Using cached narwhals-2.1.2-py3-none-any.whl.metadata (11 kB)
  Using cached gitdb-4.0.12-py3-none-any.whl.metadata (1.2 kB)
  Using cach


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [3]:
## load the trained model,scaler pickle,onehot
model=load_model('model.h5')

## load the encoder and scaler
with open('ohe.pkl','rb') as file:
      ohe=pickle.load(file)

with open('label_encoder_gender.pkl','rb') as file:
      label_encoder_gender=pickle.load(file)

with open('scaler.pkl','rb') as file:
      scaler=pickle.load(file)

In [36]:
## Example input data
input_data={
      'CreditScore':600,
      'Geography':'France',
      'Gender':'Male',
      'Age':40,
      'Tenure':3,
      'Balance':60000,
      'NumsOfproducts':2,
      'HasCrCard':1,
      'IsActiveMember':1,
      'Estimatedsalary':50000
}

In [37]:
input_data_df=pd.DataFrame([input_data])

In [38]:
## one hot encode 'geography'
geo_encoded=ohe.transform([[input_data['Geography']]]).toarray()
geo_encoded_df=pd.DataFrame(geo_encoded,columns=ohe.get_feature_names_out(['Geography']))

c:\Users\pranav\OneDrive\end-to-end_ANN_DL_project\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [39]:
## combine one-hot encoded columns with input data
input_data_df=pd.concat([input_data_df,geo_encoded_df],axis=1)

In [40]:
input_data_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumsOfproducts,HasCrCard,IsActiveMember,Estimatedsalary,Geography_France,Geography_Germany,Geography_Spain
0,600,France,Male,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [41]:
## encode categorial value

input_data_df['Gender']=label_encoder_gender.transform(input_data_df['Gender'])

In [42]:
input_data_df=input_data_df.drop('Geography',axis=1)

In [44]:
## scaling the input data
input_scaled=scaler.fit_transform(input_data_df)

In [46]:
## predict data
prediction=model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step


array([[0.2087165]], dtype=float32)

In [47]:
prediction_prob=prediction[0][0]

In [48]:
prediction_prob

np.float32(0.2087165)

In [49]:
if prediction_prob>0.5:
      print('The customer is likely to churn.')
else:
      print('The customer is not likely to churn')

The customer is not likely to churn
